In [ ]:
import sys
import scanpy as sc
import anndata
import pandas as pd
import numpy as np
import os

data_type = 'float32'

# this line forces theano to use the GPU and should go before importing cell2location
# os.environ["THEANO_FLAGS"] = 'device=cuda0,floatX=' + data_type + ',force_device=True'
# if using the CPU uncomment this:
os.environ["THEANO_FLAGS"] = 'device=cpu,floatX=float32,openmp=True,force_device=True'

import cell2location

from scipy import io
from scipy.sparse import coo_matrix, csr_matrix

import matplotlib as mpl
from matplotlib import rcParams
rcParams['pdf.fonttype'] = 42
import matplotlib.pyplot as plt
import seaborn as sns

# silence scanpy warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
sc_data_folder = '/Users/saradhamiriyala/Desktop/Cui_Lab_Bioinformatics/cell2loc/data'
results_folder = '/Users/saradhamiriyala/Desktop/Cui_Lab_Bioinformatics/cell2loc/results/P1MID7_refCOMBINED_analysis/'

if os.path.exists(results_folder) is not True:
    os.mkdir(results_folder)

# create paths and names to results folders for reference regression and cell2location models
ref_run_name = f'{results_folder}/reference_signatures'
run_name = f'{results_folder}/cell2location_map'

In [ ]:
# read in snRNA reference h5ad file
adata_snrna_raw = anndata.read_h5ad("/Users/saradhamiriyala/Desktop/Cui_Lab_Bioinformatics/cell2loc/data/ref_data_combined.h5ad")

# find mitochondria-encoded (MT) genes
adata_snrna_raw.var['MT_gene'] = [gene.startswith('mt-') for gene in adata_snrna_raw.var_names]

# remove MT genes for spatial mapping (keeping their counts in the object)
adata_snrna_raw.obsm['MT'] = adata_snrna_raw[:, adata_snrna_raw.var['MT_gene'].values].X.toarray()
adata_snrna_raw = adata_snrna_raw[:, ~adata_snrna_raw.var['MT_gene'].values]

In [ ]:
# add cell type labels as columns in adata.obs (select col index with barcode)
labels = pd.read_csv("/Users/saradhamiriyala/Desktop/Cui_Lab_Bioinformatics/cell2loc/data/ref_data_combined_metadata.csv", index_col=26)
labels = labels.reindex(index=adata_snrna_raw.obs_names)
adata_snrna_raw.obs[labels.columns] = labels
adata_snrna_raw = adata_snrna_raw[~adata_snrna_raw.obs['CellType_New'].isna(), :]

In [ ]:
from cell2location.utils.filtering import filter_genes
selected = filter_genes(adata_snrna_raw, cell_count_cutoff=5, cell_percentage_cutoff2=0.03, nonz_mean_cutoff=1.12)

# filter the object
adata_snrna_raw = adata_snrna_raw[:, selected].copy()

In [ ]:
# prepare data for the regression model
cell2location.models.RegressionModel.setup_anndata(adata=adata_snrna_raw,
                        batch_key='group',
                        labels_key='CellType_New'
                       )

In [ ]:
# create the regression model
from cell2location.models import RegressionModel
mod = RegressionModel(adata_snrna_raw)

# view anndata_setup
mod.view_anndata_setup()

In [ ]:
mod.train(max_epochs=250, use_gpu=False)

In [ ]:
mod.plot_history(20)

In [ ]:
# export the estimated cell abundance (summary of the posterior distribution).
adata_snrna_raw = mod.export_posterior(
    adata_snrna_raw, sample_kwargs={'num_samples': 1000, 'batch_size': 2500, 'use_gpu': True}
)

# Save model
mod.save(f"{ref_run_name}", overwrite=True)

# Save anndata object with results
adata_file = f"{ref_run_name}/sc.h5ad"
adata_snrna_raw.write(adata_file)
adata_file

In [ ]:
adata_snrna_raw = mod.export_posterior(
    adata_snrna_raw, use_quantiles=True,
    # choose quantiles
    add_to_varm = ["q05","q50", "q95", "q0001"],
    sample_kwargs = {'batch_size': 2500, 'use_gpu': False}
)

In [ ]:
mod.plot_QC('q50')

In [ ]:
# export estimated expression in each cluster
if 'means_per_cluster_mu_fg' in adata_snrna_raw.varm.keys():
    inf_aver = adata_snrna_raw.varm['means_per_cluster_mu_fg'][[f'means_per_cluster_mu_fg_{i}'
                                    for i in adata_snrna_raw.uns['mod']['factor_names']]].copy()
else:
    inf_aver = adata_snrna_raw.var[[f'means_per_cluster_mu_fg_{i}'
                                    for i in adata_snrna_raw.uns['mod']['factor_names']]].copy()
inf_aver.columns = adata_snrna_raw.uns['mod']['factor_names']

In [ ]:
# view estimated expression in each cluster and save as csv
inf_aver.iloc[0:5, 0:5]
inf_aver.to_csv("/Users/saradhamiriyala/Desktop/inf_aver.csv")

In [ ]:
# in read spatial sequencing data h5 file and image
adata_vis = sc.read_visium(path = '/Users/saradhamiriyala/Desktop/Cui_Lab_Bioinformatics/cell2loc/data/P1MID7',
                          count_file = 'filtered_feature_bc_matrix.h5',
                          source_image_path = '/Users/saradhamiriyala/Desktop/Cui_Lab_Bioinformatics/cell2loc/data/P1MID7/spatial/tissue_hires_image.png')
adata_vis.obs['sample'] = list(adata_vis.uns['spatial'].keys())[0]
adata_vis.var['SYMBOL'] = adata_vis.var_names
adata_vis.var.set_index(adata_vis.var_names)

# convert data type to float32
adata_vis.obs['array_row'] = adata_vis.obs['array_row'].astype('float32')
adata_vis.obs['array_col'] = adata_vis.obs['array_col'].astype('float32')
adata_vis.obs['in_tissue'] = adata_vis.obs['in_tissue'].astype('float32')
adata_vis.obsm['spatial'] = adata_vis.obsm['spatial'].astype('float32')

In [ ]:
# find shared genes and subset both anndata and reference signatures
intersect = np.intersect1d(adata_vis.var_names, inf_aver.index)
adata_vis.var_names_make_unique()
adata_vis = adata_vis[:, intersect].copy()
inf_aver = inf_aver.loc[intersect].copy()

# prepare anndata for cell2location model
cell2location.models.Cell2location.setup_anndata(adata=adata_vis, batch_key="sample")

In [ ]:
# create and train the model
mod = cell2location.models.Cell2location(
    adata_vis, cell_state_df=inf_aver,
    N_cells_per_location=10,
    detection_alpha=20
)
mod.view_anndata_setup()

In [ ]:
mod.train(max_epochs=5000,
          # train using full data (batch_size=None)
          batch_size=None,
          # use all data points in training because
          # we need to estimate cell abundance at all locations
          train_size=1,
          use_gpu=False,
         )

# plot ELBO loss history during training
mod.plot_history(20)
plt.legend(labels=['full data training']);

In [ ]:
# export the estimated cell abundance (summary of the posterior distribution).
adata_vis = mod.export_posterior(
    adata_vis, sample_kwargs={'num_samples': 1000, 'batch_size': mod.adata.n_obs, 'use_gpu': False}
)

# Save model
mod.save(f"{run_name}", overwrite=True)

# mod = cell2location.models.Cell2location.load(f"{run_name}", adata_vis)

# Save anndata object with results
adata_file = f"{run_name}/sp.h5ad"
adata_vis.write(adata_file)
adata_file

In [ ]:
mod.plot_QC()

In [ ]:
# plot QC graphs for spatial data
fig = mod.plot_spatial_QC_across_batches()

In [ ]:
# add 5% quantile, representing confident cell abundance, 'at least this amount is present',
# to adata.obs with nice names for plotting
adata_vis.obs[adata_vis.uns['mod']['factor_names']] = adata_vis.obsm['q05_cell_abundance_w_sf']

# select one slide
from cell2location.utils import select_slide
slide = select_slide(adata_vis, 'P1MID7')

# plot in spatial coordinates
with mpl.rc_context({'axes.facecolor':  'black',
                     'figure.figsize': [4.5, 5]}):

    sc.pl.spatial(slide, cmap='magma',
                  color=["B Cells", "Blood Cells", "CM1", "CM2", "CM3", "CM4", "CM5",
                         "EC", "EPI", "EndoEC", "FB", "Glial", 
                         "Macrophage", "Pericyte/SMC", "T cells"],
                  ncols=4, size=1.3,
                  img_key='hires',
                  # limit color scale at 99.2% quantile of cell abundance
                  vmin=0, vmax='p99.2',
                  save=True
                 )